# Jupyter Notebook and IPython
## Where Python meets the Web

### Based on:
![Cookbook](images/cover-cookbook-2nd.png)
http://ipython-books.github.io/

## First some cheap tricks

### Bash Support

In [ ]:
!ls

### Line Magics

In [ ]:
%lsmagic

### Built-in docs

In [ ]:
%run?

## We can craft our own magics

In [ ]:
from IPython.core.magic import (register_line_magic,
                                register_cell_magic)

In [ ]:
@register_line_magic
def hello(line):
    if line == 'french':
        print("Salut tout le monde!")
    else:
        print("Hello world!")

In [ ]:
%hello

In [ ]:
%hello french

### Something a bit more useful

In [ ]:
import pandas as pd
from io import StringIO

@register_cell_magic
def csv(line, cell):
    # We create a string buffer containing the
    # contents of the cell.
    sio = StringIO(cell)
    # We use Pandas' read_csv function to parse
    # the CSV string.
    return pd.read_csv(sio)

In [ ]:
%%csv
col1,col2,col3
0,1,2
3,4,5
7,8,9

In [ ]:
df = _
df.describe()

### We can also package magics

In [ ]:
%%writefile csvmagic.py
import pandas as pd
from io import StringIO

def csv(line, cell):
    sio = StringIO(cell)
    return pd.read_csv(sio)

def load_ipython_extension(ipython):
    """This function is called when the extension is
    loaded. It accepts an IPython InteractiveShell
    instance. We can register the magic with the
    `register_magic_function` method of the shell
    instance."""
    ipython.register_magic_function(csv, 'cell')

In [ ]:
%load_ext csvmagic

In [ ]:
%%csv
col1,col2,col3
0,1,2
3,4,5
7,8,9

## Wait, we're in a browser

In [ ]:
from IPython.display import HTML
HTML('''
<table style="border: 2px solid black;">
''' +
     ''.join(['<tr>' +
              ''.join([f'<td>{row},{col}</td>'
                       for col in range(5)]) +
              '</tr>' for row in range(5)]) +
     '''
</table>
''')

In [ ]:
from IPython.display import SVG
SVG('''<svg width="600" height="80">''' +
    ''.join([f'''<circle
              cx="{(30 + 3*i) * (10 - i)}"
              cy="30"
              r="{3. * float(i)}"
              fill="red"
              stroke-width="2"
              stroke="black">
        </circle>''' for i in range(10)]) +
    '''</svg>''')

In [ ]:
from IPython.display import YouTubeVideo
YouTubeVideo('VQBZ2MqWBZI')

## And we have the best Data Analysis tools

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

### So let's use them

In [ ]:
url = ("https://raw.githubusercontent.com/"
       "ipython-books/cookbook-2nd-data/"
       "master/bikes.csv")
url = 'data/comptagesvelo2015.csv'

In [ ]:
df = pd.read_csv(url, index_col='Date',
                 parse_dates=True, dayfirst=True)

In [ ]:
df.head(2)

In [ ]:
df.describe()

### Simple plotting

In [ ]:
df[['Berri1', 'PierDup']].plot(figsize=(10, 6),
                               style=['-', '--'],
                               lw=2)

### Time-series analysis

In [ ]:
df.index.weekday_name

In [ ]:
df_week = df.groupby(df.index.weekday).sum()

In [ ]:
df_week

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 8))
df_week.plot(style='-o', lw=3, ax=ax)
ax.set_xlabel('Weekday')
# We replace the labels 0, 1, 2... by the weekday names.
ax.set_xticklabels(
    ('Monday,Tuesday,Wednesday,Thursday,'
     'Friday,Saturday,Sunday').split(','))
ax.set_ylim(0)  # Set the bottom axis to 0.

### Some human interaction

In [ ]:
from ipywidgets import interact

@interact
def plot(n=(1, 30)):
    fig, ax = plt.subplots(1, 1, figsize=(10, 8))
    df['Berri1'].rolling(window=n).mean().plot(ax=ax)
    ax.set_ylim(0, 7000)
    plt.show()

## Re-using web frameworks

In [ ]:
import numpy as np
import pandas as pd
import bokeh
import bokeh.plotting as bkh
bkh.output_notebook()

In [ ]:
f = bkh.figure(width=600, height=400)
f.circle(np.random.randn(1000),
         np.random.randn(1000),
         size=np.random.uniform(2, 10, 1000),
         alpha=.5)
bkh.show(f)

In [ ]:
from ipyleaflet import Map, Marker

pos = [32.08, 34.8]
m = Map(center=pos, zoom=17)
marker = Marker(location=pos,
                rise_on_hover=True,
                title="Here I am!",
                )
m += marker
m

## More widgets, please

In [ ]:
import ipywidgets as widgets
from ipywidgets import HBox, VBox
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
%matplotlib inline

In [ ]:
@widgets.interact
def f(x=5):
    print(x)

In [ ]:
@widgets.interact(x=(0, 5))
def f(x=5):
    print(x)

In [ ]:
@widgets.interact_manual(
    color=['blue', 'red', 'green'], lw=(1., 10.))
def plot(freq=1., color='blue', lw=2, grid=True):
    t = np.linspace(-1., +1., 1000)
    fig, ax = plt.subplots(1, 1, figsize=(8, 6))
    ax.plot(t, np.sin(2 * np.pi * freq * t),
            lw=lw, color=color)
    ax.grid(grid)

In [ ]:
freq_slider = widgets.FloatSlider(
    value=2.,
    min=1.,
    max=10.0,
    step=0.1,
    description='Frequency:',
    readout_format='.1f',
)
freq_slider

In [ ]:
range_slider = widgets.FloatRangeSlider(
    value=[-1., +1.],
    min=-5., max=+5., step=0.1,
    description='xlim:',
    readout_format='.1f',
)
range_slider

In [ ]:
grid_button = widgets.ToggleButton(
    value=False,
    description='Grid',
    icon='check'
)
grid_button

In [ ]:
color_buttons = widgets.ToggleButtons(
    options=['blue', 'red', 'green'],
    description='Color:',
)
color_buttons

In [ ]:
title_textbox = widgets.Text(
    value='Hello World',
    description='Title:',
)
title_textbox

In [ ]:
color_picker = widgets.ColorPicker(
    concise=True,
    description='Background color:',
    value='#efefef',
)
color_picker

In [ ]:
button = widgets.Button(
    description='Plot',
)
button

In [ ]:
def plot2(b=None):
    xlim = range_slider.value
    freq = freq_slider.value
    grid = grid_button.value
    color = color_buttons.value
    title = title_textbox.value
    bgcolor = color_picker.value

    t = np.linspace(xlim[0], xlim[1], 1000)
    f, ax = plt.subplots(1, 1, figsize=(8, 6))
    ax.set_title(title)
    ax.set_facecolor(bgcolor)
    ax.plot(t, np.sin(2 * np.pi * freq * t), 
            color=color)
    ax.grid(grid)

In [ ]:
@button.on_click
def plot_on_click(b):
    plot2()

## Let's bring it all together

In [ ]:
tab1 = VBox(children=[freq_slider,
                      range_slider,
                      ])
tab2 = VBox(children=[color_buttons,
                      HBox(children=[title_textbox,
                                     color_picker,
                                     grid_button]),
                                     ])

In [ ]:
tab = widgets.Tab(children=[tab1, tab2])
tab.set_title(0, 'plot')
tab.set_title(1, 'styling')
VBox(children=[tab, button])

### We can even define new widgets in-line

In [ ]:
import ipywidgets as widgets
from traitlets import Unicode, Int, validate

In [ ]:
class CounterWidget(widgets.DOMWidget):
    _view_name = Unicode('CounterView').tag(sync=True)
    _view_module = Unicode('counter').tag(sync=True)
    value = Int(0).tag(sync=True)

In [ ]:
%%javascript
// We make sure the `counter` module is defined
// only once.
require.undef('counter');

// We define the `counter` module depending on the
// Jupyter widgets framework.
define('counter', ["@jupyter-widgets/base"],
       function(widgets) {

    // We create the CounterView frontend class,
    // deriving from DOMWidgetView.
    var CounterView = widgets.DOMWidgetView.extend({

        // This method creates the HTML widget.
        render: function() {
            // The value_changed() method should be
            // called when the model's value changes
            // on the kernel side.
            this.value_changed();
            this.model.on('change:value',
                          this.value_changed, this);

            var model = this.model;
            var that = this;

            // We create the plus and minus buttons.
            this.bm = $('<button/>')
            .text('-')
            .click(function() {
                // When the button is clicked,
                // the model's value is updated.
                var x = model.get('value');
                model.set('value', x - 1);
                that.touch();
            });

            this.bp = $('<button/>')
            .text('+')
            .click(function() {
                var x = model.get('value');
                model.set('value', x + 1);
                that.touch();
            });

            // This element displays the current
            // value of the counter.
            this.span = $('<span />')
            .text('0')
            .css({marginLeft: '10px',
                  marginRight: '10px'});

            // this.el represents the widget's DOM
            // element. We add the minus button,
            // the span element, and the plus button.
            $(this.el)
            .append(this.bm)
            .append(this.span)
            .append(this.bp);
        },

        value_changed: function() {
            // Update the displayed number when the
            // counter's value changes.
            var x = this.model.get('value');
            $($(this.el).children()[1]).text(x);
        },
    });

    return {
        CounterView : CounterView
    };
});

In [ ]:
w = CounterWidget()
w

In [ ]:
print(w.value)